In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')
base = 'drive/MyDrive/data/sirius'

Mounted at /content/drive


In [4]:
user = pd.read_csv(base + '/user_2.csv')
user.head()

,id,gender,birthday,region,last_visited
0,1,М,2007-07-15 18:53:02,Город Москва,2023-09-20 18:00:34.464576
1,2,Ж,2007-06-16 11:48:48,Краснодарский край,2023-10-04 16:01:11.464583
2,3,М,2010-08-09 14:11:20,Город Санкт-Петербург,2023-09-27 17:14:43.464586
3,4,М,2008-07-03 10:50:41,Город Санкт-Петербург,2023-09-27 13:46:11.464589
4,5,М,2010-02-15 23:03:02,Краснодарский край,2023-10-12 10:16:43.464591


In [5]:
user_session = pd.read_csv(base + '/user_session.csv')
user_session.head()

,session_id,user_id,date,time_watched
0,1,558,2023-01-23 00:52:14,10
1,2,435,2023-01-06 20:59:45,51
2,3,783,2023-01-27 19:24:25,2
3,4,575,2023-01-23 21:55:54,29
4,5,598,2023-01-22 15:06:09,5


## Задача 1

Посчитайте долю пользователей мужского пола в таблице `user`.

Ответ округлите до $2$-х знаков после запятой.

**Рекомендации по решению**

Для подсчёта доли объектов, удовлетворяющих заданному условию, удобно использовать функцию `mean`, вызвав её от колонки-фильтра со значениями булевого типа. В данном случае каждое знаечние `True` будет приведено к $1$, а каждое значение `False` — к $0$. Усреднение этих значений как раз и даст долю объектов, удволетворяющих заданному условию.

In [6]:
len(user[user['gender'] == 'М'])  / len(user)

0.425

## Задача 2

Посчитайте долю пользователей из Краснодарского края, которые заходили на платформу в сентябре $2023$-го года.

Ответ округлите до $2$-х знаков после запятой.

In [7]:
user['last_visited'] = pd.to_datetime(user['last_visited'], format='%Y-%m-%d %H:%M:%S.%f')

In [8]:
len(
    user[
        (user['region'] == 'Краснодарский край') &
         (user['last_visited'].dt.month == 9) &
          (user['last_visited'].dt.year == 2023)
     ]
) / len(user[user['region'] == 'Краснодарский край'])

0.41007194244604317

## Задача 3

По таблице `user_session` рассчитайте долю активных пользователей в мае $2023$-го года относительно общего числа пользователей платформы из таблицы `user`.

Активными считайте пользователей, у которых в этот месяц была хотя бы одна сессия.

Ответ округлите до $2$-х знаков после запятой.

In [9]:
user_session['date'] = pd.to_datetime(user_session['date'], format='%Y-%m-%d %H:%M:%S')

In [10]:
may_2023_user_session = user_session \
 [(user_session['date'].dt.month == 5) & (user_session['date'].dt.year == 2023)] \
 .groupby('user_id') \
 .agg({'session_id': 'count'}) \
 .rename(columns={'session_id': 'sessions_count'}) \
 .query('sessions_count > 0')
len(may_2023_user_session ) / len(user)

0.598

## Задача 4

Дадим другое определение активным пользователям. Будем считать активными в конкретном месяце тех, кто посмотрел хотя бы $60$ минут видеоконтента в этом месяце.

Рассчитайте долю активных пользователей среди тех, у кого была хотя бы одна сессия в марте $2023$-го года.

Ответ округлите до $2$-х знаков после запятой.

In [11]:
march_2023_user_session = user_session \
 [(user_session['date'].dt.month == 3) & (user_session['date'].dt.year == 2023)] \
.groupby('user_id') \
.agg({'session_id': 'count', 'time_watched': 'sum'}) \
.rename(columns={'session_id': 'session_count'}) \
.query('session_count > 0')

march_2023_active_user_session = march_2023_user_session[march_2023_user_session['time_watched'] >= 60]

len(march_2023_active_user_session) / len(march_2023_user_session)

0.9192872117400419

# Задачи 5-9

Рассчитайте, сколько в среднем смотрят видеоконтента пользователи за один день активности. Днём активности называем день, в который у пользователя была сессия.

## Задача 5

Для этого сначала вычислите суммарное число дней, когда пользователи смотрели видеоконтент.

Обратите внимание, что хараткеристику можно посчитать двумя способами. Рассчитатйте её как число уникальных пар вида (пользователь; день, когда у него была сессия).

In [15]:
user_session_modified = user_session.copy()
user_session_modified['date'] = user_session_modified['date'].dt.date
user_session_modified = user_session_modified.groupby('date', as_index=False).agg({'user_id': 'count'})
user_session_modified['user_id'].count()

## Задача 6

Теперь вычислите суммарное время просмотра видеоконтента пользователями.

## Задача 7

Вычислите среднее количество просмотренного пользователями видеоконтента за день активности, как отношение суммарного времени просмотра к суммарному числу дней просмотра.

Ответ округлите до $2$-х знаков после запятой.

## Задача 8

Обратите внимание, что есть альтернативный способ рассчитать, сколько в среднем видеоконтента смотрят пользователи за один день активности.

Во-первых, рассчитайте для каждого из пользователей среднее количество просмотренного им видеоконтента. Принцип вычисления метрики для конкретного пользователя такой же, как был для всех пользователей вместе: нужно взять суммарное время просмотра контента и разделить на суммарное количество уникальных дней активности.

В качестве ответа укажите среднее количество просмотренного видеоконтента пользователем с `id`, равным $738$. Ответ округлите до $2$-х знаков после запятой.

**Рекомендации по решению**

Для того чтобы вычислить среднее количество видеоконтента, просмотренного конкретным пользователем, удобно провести группировку по пользователям и произвести агрегацию по двум колонкам отдельно: по колонке `time_watched`, просуммировав значения в ней , и по колонке `date`, посчитав количество уникальных дней активности для каждого из пользователей.

Для того чтобы произвести агрегацию по колонке `date`, удобно воспользоваться лямбда-функцией, которая сначала произведёт округление моментов времени из колонке `date` _вниз_ до дней, а потом посчитает количество полученных уникальных дней.

## Задача 9

Для того чтобы вычислить среднее время просмотра видеоконтента за день активности, посчитайте среднее арифметическое данной характеристики по всем пользователям. Для этого используйте данные, полученные в предыдущем упражнении.

Ответ округлите до $2$-х знаков после запятой.